## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-13-14-49-51 +0000,wapo,Israel begins releasing Palestinian detainees ...,https://www.washingtonpost.com
1,2025-10-13-14-49-51 +0000,wapo,"Live updates: Trump says war in Gaza ‘is over,...",https://www.washingtonpost.com/world/2025/10/1...
2,2025-10-13-14-46-27 +0000,nyt,Live Updates: Hostages and Palestinian Prisone...,https://www.nytimes.com/live/2025/10/12/world/...
3,2025-10-13-14-45-48 +0000,nyt,What We Know About the Deal Between Israel and...,https://www.nytimes.com/2025/10/09/world/middl...
4,2025-10-13-14-45-04 +0000,startribune,Trump arrives in Egypt for Gaza summit after u...,https://www.startribune.com/trump-arriving-in-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,44
0,israel,22
40,china,21
15,hostages,15
23,hamas,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
1,2025-10-13-14-49-51 +0000,wapo,"Live updates: Trump says war in Gaza ‘is over,...",https://www.washingtonpost.com/world/2025/10/1...,127
196,2025-10-12-22-52-05 +0000,nypost,Hillary Clinton heaps rare praise on President...,https://nypost.com/2025/10/12/us-news/hillary-...,118
149,2025-10-13-05-31-15 +0000,nypost,‘War is over’ Trump declares — as he arrives i...,https://nypost.com/2025/10/13/world-news/war-i...,110
4,2025-10-13-14-45-04 +0000,startribune,Trump arrives in Egypt for Gaza summit after u...,https://www.startribune.com/trump-arriving-in-...,110
139,2025-10-13-07-20-04 +0000,nypost,"What Trump told Netanyahu, father of ambassado...",https://nypost.com/2025/10/13/world-news/trump...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
1,127,2025-10-13-14-49-51 +0000,wapo,"Live updates: Trump says war in Gaza ‘is over,...",https://www.washingtonpost.com/world/2025/10/1...
235,59,2025-10-12-17-31-50 +0000,nypost,Father of Israeli-American hostage assumed mur...,https://nypost.com/2025/10/12/world-news/hosta...
236,50,2025-10-12-17-23-38 +0000,nyt,Cars to Fighter Jets: China’s New Export Curbs...,https://www.nytimes.com/2025/10/12/business/ch...
64,50,2025-10-13-11-28-00 +0000,wsj,"Joel Mokyr, Philippe Aghion and Peter Howitt w...",https://www.wsj.com/economy/nobel-economics-pr...
145,38,2025-10-13-06-16-39 +0000,nypost,Last living hostages all released after 738 da...,https://nypost.com/2025/10/13/world-news/last-...
114,32,2025-10-13-09-06-41 +0000,nypost,Heartwarming moment shows Israeli hostage spea...,https://nypost.com/2025/10/13/world-news/mothe...
155,28,2025-10-13-04-28-00 +0000,wsj,Asia Stock Markets Fall as U.S.-China Trade Te...,https://www.wsj.com/finance/stocks/asia-stock-...
144,28,2025-10-13-06-21-00 +0000,bbc,"Boy, 11, shoots farm worker dead in tragic Sou...",https://www.bbc.com/news/articles/ce86dvr03zqo...
38,27,2025-10-13-13-06-20 +0000,nypost,Trump urges Israeli president to pardon Benjam...,https://nypost.com/2025/10/13/us-news/trump-ur...
124,24,2025-10-13-08-37-59 +0000,bbc,Watch: Emotional moment mum video calls hostag...,https://www.bbc.com/news/videos/c33pn1lrk0ro?a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
